# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [35]:
av_df=pd.read_csv('AviationData.csv', index_col=0, encoding='latin1', low_memory=False)

In [36]:
av_df.head()

,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,Injury.Severity,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
Event.Id,,,,,,,,,,,,,,,,,,,,,
20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,Fatal(2),...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,Fatal(4),...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,Fatal(3),...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,Fatal(2),...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,Fatal(1),...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [37]:
av_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88889 entries, 20001218X45444 to 20221230106513
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Investigation.Type      88889 non-null  object 
 1   Accident.Number         88889 non-null  object 
 2   Event.Date              88889 non-null  object 
 3   Location                88837 non-null  object 
 4   Country                 88663 non-null  object 
 5   Latitude                34382 non-null  object 
 6   Longitude               34373 non-null  object 
 7   Airport.Code            50132 non-null  object 
 8   Airport.Name            52704 non-null  object 
 9   Injury.Severity         87889 non-null  object 
 10  Aircraft.damage         85695 non-null  object 
 11  Aircraft.Category       32287 non-null  object 
 12  Registration.Number     87507 non-null  object 
 13  Make                    88826 non-null  object 
 14  Model                

In [38]:
av_df.shape

(88889, 30)

In [39]:
len(av_df.Country.unique()) #number of crountries
#av_df.Country.unique()

220

Looking for duplicated rows

In [40]:
av_df.duplicated().sum()

0

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

Removing rows with Event.Date older than **1983**. Since the max life expected of an aircraft is only ~40 years, therefore the  older models are not useful to us in this analysis.

In [41]:
#convert Event.Date to datetime format
av_df['Event.Date'] = pd.to_datetime(av_df['Event.Date'], errors='coerce')

#filter to include rows from 1/1/1983 and onward
av_df = av_df[av_df['Event.Date'] >= '1983-01-01']
av_df.shape
print(av_df['Event.Date'].min(), av_df['Event.Date'].max())

1983-01-01 00:00:00 2022-12-29 00:00:00


In [42]:
av_df.shape

(85289, 30)

That removed 3,600 rows from the Data Frame

Exploring Amateur.Built column

In [43]:
av_df['Amateur.Built'].unique()

array(['No', 'Yes', nan], dtype=object)

In [44]:
av_df['Amateur.Built'].value_counts(dropna=False)

Amateur.Built
No     76960
Yes     8228
NaN      101
Name: count, dtype: int64

**Removing blank rows or rows containing 'Yes'** to eliminate non-professional builds from the dataset

In [45]:
av_df = av_df[
    (av_df['Amateur.Built'] != 'Yes') & (av_df['Amateur.Built'].notna())]

av_df['Amateur.Built'].value_counts() #checking they were removed

Amateur.Built
No    76960
Name: count, dtype: int64

Explore what type of aircrafts are in this dataset.

In [46]:
av_df['Aircraft.Category'].unique()

array([nan, 'Glider', 'Airplane', 'Helicopter', 'Blimp', 'Balloon',
       'Gyrocraft', 'Ultralight', 'Powered-Lift', 'Weight-Shift',
       'Powered Parachute', 'Unknown', 'Rocket', 'WSFT', 'UNK'],
      dtype=object)

In [47]:
av_df['Aircraft.Category'].value_counts(dropna=False)

Aircraft.Category
NaN                  51555
Airplane             21447
Helicopter            3022
Glider                 434
Balloon                199
Weight-Shift           139
Powered Parachute       83
Gyrocraft               31
Ultralight              18
Unknown                 11
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
Name: count, dtype: int64

Removing **noncommercial and nonprofessional** aircraft from dataset.

In [48]:
#av_df=av_df.query("Aircraft.Category=='Airplane' or Aircraft.Category=='Helicopter'").copy()
av_df=av_df.query("`Aircraft.Category` == 'Airplane' or `Aircraft.Category` == 'Helicopter'")

In [49]:
av_df['Aircraft.Category'].value_counts()

Aircraft.Category
Airplane      21447
Helicopter     3022
Name: count, dtype: int64

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

Exploring the 4 coulmns with containing numbers for injuried and uninjuried, it can be assumed that and NaNs can be filled with **0** instead of being blank.

In [69]:
#av_df['Total.Fatal.Injuries'].unique()
#av_df['Total.Serious.Injuries'].unique()
av_df['Total.Minor.Injuries'].unique()
#av_df['Total.Uninjured'].unique()

array([  0.,   1.,   4.,   3.,   2.,  12.,   9.,   6., 200.,   8.,   5.,
        47.,  10.,  42.,  13.,   7.,  40.,  18.,  22.,  27.,  17.,  50.,
        14.,  16.,  19.])

In [74]:
#changing Nan's to 0
injury_cols=['Total.Fatal.Injuries','Total.Serious.Injuries','Total.Minor.Injuries','Total.Uninjured']

av_df[injury_cols] = av_df[injury_cols].apply(pd.to_numeric, errors='coerce')

av_df[injury_cols] = av_df[injury_cols].fillna(0)

In [75]:
#Double checking it worked
av_df['Total.Fatal.Injuries'].unique()
#av_df['Total.Uninjured'].unique()

array([  0.,   1.,  11.,  17.,   3.,   2.,  10.,   7.,   4.,  27.,  16.,
        12.,   5.,  14.,  54.,   6., 160.,  97., 125., 228.,   9.,   8.,
        13.,  18., 169., 131.,  24.,  20.,  65.,  19.,  26., 113., 154.,
        30.,  88.,  49., 152.,  90.,  89., 103., 158., 157.,  42.,  21.,
        77., 127.,  44.,  50.,  33., 239., 295.,  58., 162.,  43., 150.,
       224.,  23.,  62.,  66.,  71., 112., 188.,  41., 176., 132.])

Calculating the total passangers on the flight per listing (for each row)
-This is assuming the Total.Uninjured column is giving correct values

In [77]:
av_df['Total.Passengers']=(av_df['Total.Fatal.Injuries'] + av_df['Total.Serious.Injuries'] 
                         + av_df['Total.Minor.Injuries'] + av_df['Total.Uninjured'])
print(av_df.columns)

Index(['Investigation.Type', 'Accident.Number', 'Event.Date', 'Location',
       'Country', 'Latitude', 'Longitude', 'Airport.Code', 'Airport.Name',
       'Injury.Severity', 'Aircraft.damage', 'Aircraft.Category',
       'Registration.Number', 'Make', 'Model', 'Amateur.Built',
       'Number.of.Engines', 'Engine.Type', 'FAR.Description', 'Schedule',
       'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Total.Passangers', 'Weather.Condition', 'Broad.phase.of.flight',
       'Report.Status', 'Publication.Date', 'Total.Passengers'],
      dtype='object')


Calculate the injury rate of **Fatal + Serious**, I've chosen not to include minor injuries b/c when I looked throught the dataset, on a handfull incidents have large numbers.  Also, I am making the assumption that an insurance company is not worried about minor injuries, but major ones where they could get sued.

In [78]:
av_df['Fatal.Serious.Injury.Rate']=np.where(av_df['Total.Passengers'] > 0,
    (av_df['Total.Fatal.Injuries'] + av_df['Total.Serious.Injuries'])/av_df['Total.Passengers'], np.nan)

Now, I am changing the number values and **binning** them to make the more categorical

In [80]:
#Binning the data
def categorize_risk(rate):
    if pd.isna(rate):
        return 'Unknown'
    elif rate == 0:
        return 'No Risk'
    elif rate <= 0.25:
        return 'Low Risk'
    elif rate <= 0.75:
        return 'Moderate Risk'
    else:
        return 'High Risk'

av_df['Injury.Risk.Category']=av_df['Fatal.Serious.Injury.Rate'].apply(categorize_risk)

In [81]:
av_df['Injury.Risk.Category'].value_counts()

Injury.Risk.Category
No Risk          15493
High Risk         6052
Moderate Risk     1596
Unknown            976
Low Risk           352
Name: count, dtype: int64

So I added 3 columns to the dataset, 'Total.Passengers','Fatal.Serious.Injury.Rate', 'Injury.Risk.Category' ~~Wooo

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [88]:
av_df['Aircraft.damage'].unique()

array(['Minor', 'Destroyed', nan, 'Substantial', 'Unknown'], dtype=object)

Lets see what the catagories are for Aircraft.damage

In [89]:
av_df['Aircraft.damage'].value_counts(dropna=False)

Aircraft.damage
Substantial    19414
Destroyed       2772
NaN             1296
Minor            875
Unknown          112
Name: count, dtype: int64

Removing rows from Aircraft.damage that contain Unknown or are blank.  Keeping row that contain **Destroyed, Substaintial, or Minor.**

In [92]:
av_df = av_df.query("`Aircraft.damage` in ['Destroyed', 'Substantial', 'Minor']")

#removing rows not containing the 3 levels of damage
av_df = av_df[av_df['Aircraft.damage'].isin(['Destroyed', 'Substantial', 'Minor'])]

av_df['Aircraft.damage'].value_counts()

Aircraft.damage
Substantial    19414
Destroyed       2772
Minor            875
Name: count, dtype: int64

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [94]:
av_df['Make'].unique()

array(['Lockheed', 'Boeing', 'Piper', ..., 'PHANTOM', 'JAMES R DERNOVSEK',
       'ORLICAN S R O'], dtype=object)

Drop missing or unknown rows from Make column

In [99]:
#NaN
av_df = av_df[av_df['Make'].notna()]

#Eliminate white spaces and change all letters to uppercase to find all the unknowns
#Eliminate such things as corp. vs corp
av_df['Make'] = av_df['Make'].str.strip().str.upper().str.replace('.', '', regex=False)

#remove all unknowns
av_df = av_df[av_df['Make'] != 'UNKNOWN']

In [100]:
av_df['Make'].shape

(23058,)

Remove makes with less than 50 inccidents of accidents.

In [101]:
make_counts = av_df['Make'].value_counts()
valid_makes = make_counts[make_counts >= 50].index
av_df = av_df[av_df['Make'].isin(valid_makes)]

In [102]:
av_df['Make'].shape

(19154,)

### Inspect Model column
- Get rid of any **NaNs**.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [113]:
av_df['Model'].value_counts()

Model
172          752
152          309
182          300
R44          295
172S         269
            ... 
AT-6G          1
BF12           1
S-55           1
PA-46-31P      1
PA-44          1
Name: count, Length: 2199, dtype: int64

Drop missing or unknown rows from Model column

In [114]:
av_df = av_df.dropna(subset=['Model'])
av_df['Model'].value_counts()

Model
172          752
152          309
182          300
R44          295
172S         269
            ... 
AT-6G          1
BF12           1
S-55           1
PA-46-31P      1
PA-44          1
Name: count, Length: 2199, dtype: int64

In [108]:
#make all uppercase to find all unkowns
av_df['Make'] = av_df['Make'].str.strip().str.upper()

#remove all unknowns
av_df = av_df[av_df['Make'] != 'UNKNOWN']

Checking if model names used by more than 1 manufacturer

In [118]:
av_df['model.make.counts']=av_df.groupby('Model')['Make'].nunique()

Event.Id
20001214X42478   NaN
20001214X42331   NaN
20001214X45013   NaN
20001214X45188   NaN
20001214X45339   NaN
                  ..
20221212106444   NaN
20221213106455   NaN
20221215106463   NaN
20221219106470   NaN
20221227106497   NaN
Name: model.make.counts, Length: 19141, dtype: float64

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [ ]:
Checking to see which columns contain the most **blanks cells**

In [ ]:
col_percent_missing=av_df.isna().mean() * 100
print(col_percent_missing.sort_values(ascending=False))

In [ ]:
drop log, lat, airport.code

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [ ]:
av_df.to_csv('Cleaned_AviationData.csv', index=False)